In [1]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.decomposition import PCA
from nltk.corpus import stopwords
import pickle
import pandas as pd
from bertopic.vectorizers import ClassTfidfTransformer
import plotly.io as pio

from tqdm._tqdm_notebook import tqdm

import openai

from bertopic.representation import KeyBERTInspired

from transformers import pipeline
from bertopic.representation import TextGeneration
from bertopic.representation import PartOfSpeech
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import OpenAI

/opt/homebrew/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/lib/python3.11/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keywo

In [2]:
# this functions are only to save the embedding
def save(x, file_name):
    with open(file_name, 'wb') as handle:
        pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)

def restore(file_name):
    with open(file_name, 'rb') as handle:
        x = pickle.load(handle)
    return x

In [3]:
df = pd.read_table('../data/biology_abstracts.csv',sep=';')

Prepare data

In [4]:
df['text'] = df.title+ ' ' + df.abstract
df['text'] = df.text.fillna('')

In [5]:
text_df = df[-df.OST_BK.duplicated(keep='last')].reset_index(drop=True) #drop duplicates

In [6]:
# umap_model = UMAP(n_neighbors=15, n_components=3, min_dist=0.0, metric='cosine', low_memory = True, n_jobs=32)
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2") #"paraphrase-MiniLM-L3-v2"

def rescale(x, inplace=False):
    """ Rescale an embedding so optimization will not have convergence issues.
    """
    if not inplace:
        x = np.array(x, copy=True)

    x /= np.std(x[:, 0]) * 10000

    return x

In [7]:
# %%time
# embeddings = sentence_model.encode(text_df.text.values, show_progress_bar=False)
# save(embeddings, '../results/embeddings_1.p')

In [8]:
embeddings = restore('../results/embeddings_1.p')

In [9]:
embeddings.shape

(34797, 384)

In [10]:
pca_embeddings = rescale(PCA(n_components=5).fit_transform(embeddings))

In [11]:
pca_embeddings.shape

(34797, 5)

## basic elements

In [12]:
umap_model = UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
    metric="cosine",
    init=pca_embeddings,
    random_state=1234, n_jobs=4, low_memory=False)    
sw=stopwords.words(['english','spanish','french']) + ['elsevier']
vectorizer_model = CountVectorizer(ngram_range=(1,1), stop_words=sw,max_df=1.0, min_df=0.001)
# Setting HDBSCAN model
hdbscan_model = HDBSCAN(min_cluster_size=100,min_samples=1, metric='euclidean',cluster_selection_epsilon=0.05, 
                        cluster_selection_method='leaf', prediction_data=True,core_dist_n_jobs=4,memory='tmp/') #, min_samples=1
ctfidf_model  = ClassTfidfTransformer()

## Representation models

In [13]:
# The main representation of a topic
main_representation = KeyBERTInspired()

### LLM


In [18]:
# client = openai.OpenAI(api_key="")


In [19]:
# Create your representation model
short_name_prompt = """
I have a corpus of Biology with 100 topics. I have a topic that is described by the following keywords: [KEYWORDS]
Based on the information above, extract a short topic label of a signle word that can accurately represent the topic, in the following format:
topic: <topic label>
"""
long_name_prompt = """
I have a corpus of Biology with 100 topics. I have a topic that is described by the following keywords: [KEYWORDS]
Based on the information above, extract a short topic label between one and three words that can accurately represent the topic, in the following format:
topic: <topic label>
"""
# title_prompt = """
# I have a topic that is described by the following keywords: [KEYWORDS]
# Based on the previous keywords, please give me a title that can describe the topic
# """

generator = pipeline('text2text-generation', model='google/flan-t5-base')

flan_snp = TextGeneration(generator,prompt=short_name_prompt)
flan_lnp = TextGeneration(generator,prompt=long_name_prompt)

openai4m_snp = OpenAI(client, model="gpt-4o-mini", chat=True, prompt=short_name_prompt, nr_docs=50, delay_in_seconds=1)
openai4m_lnp = OpenAI(client, model="gpt-4o-mini", chat=True, prompt=long_name_prompt, nr_docs=50, delay_in_seconds=1)
# openai4m_tp = OpenAI(client, model="gpt-4o-mini", chat=True, prompt=title_prompt, nr_docs=50, delay_in_seconds=1)

openai4o_snp = OpenAI(client, model="gpt-4o", chat=True, prompt=short_name_prompt, nr_docs=50, delay_in_seconds=1)
openai4o_lnp = OpenAI(client, model="gpt-4o", chat=True, prompt=long_name_prompt, nr_docs=50, delay_in_seconds=1)
# openai4o_tp = OpenAI(client, model="gpt-4o", chat=True, prompt=title_prompt, nr_docs=50, delay_in_seconds=1)



In [20]:
# Add all models together to be run in a single `fit`
representation_models = {
   "Main": main_representation,
    "Aspect1": flan_snp,
    "Aspect2": flan_lnp,
    "Aspect3": openai4m_snp,
    "Aspect4": openai4m_lnp,
    "Aspect5": openai4o_snp,
    "Aspect6": openai4o_lnp
}


#  Model FIT

In [21]:

topic_model = BERTopic(verbose=True,embedding_model=sentence_model, low_memory=True, calculate_probabilities=False, 
                       vectorizer_model=vectorizer_model, 
                       hdbscan_model=hdbscan_model,
                       umap_model=umap_model,
                       ctfidf_model=ctfidf_model,
                       representation_model=representation_models)


In [43]:
%%time
topics, probabilities  = topic_model.fit_transform(text_df.text.values, embeddings)

2024-08-05 14:37:57,298 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-08-05 14:38:11,765 - BERTopic - Dimensionality - Completed ✓
2024-08-05 14:38:11,766 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-05 14:38:11,902 - BERTopic - Cluster - Completed ✓
2024-08-05 14:38:11,906 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-08-05 14:38:13,822 - BERTopic - Representation - Completed ✓


CPU times: user 20 s, sys: 86.2 ms, total: 20.1 s
Wall time: 17.9 s


In [34]:
topic_info= topic_model.get_topic_info()

topic_info['Aspect1'] = topic_info['Aspect1'].apply(lambda x: x[0] if x else '').str.replace("'", "").str.replace('"', '')
topic_info['Aspect2'] = topic_info['Aspect2'].apply(lambda x: x[0] if x else '').str.replace("'", "").str.replace('"', '')
topic_info['Aspect3'] = topic_info['Aspect3'].apply(lambda x: x[0] if x else '').str.replace("'", "").str.replace('"', '')
topic_info['Aspect4'] = topic_info['Aspect4'].apply(lambda x: x[0] if x else '').str.replace("'", "").str.replace('"', '')
topic_info['Aspect5'] = topic_info['Aspect5'].apply(lambda x: x[0] if x else '').str.replace("'", "").str.replace('"', '')
topic_info['Aspect6'] = topic_info['Aspect6'].apply(lambda x: x[0] if x else '').str.replace("'", "").str.replace('"', '')


In [36]:
topic_info= topic_info.rename(columns={
    "Aspect1": "flan_snp",
    "Aspect2": "flan_lnp",
    "Aspect3": "openai4m_snp",
    "Aspect4": "openai4m_lnp",
    "Aspect5": "openai4o_snp",
    "Aspect6": "openai4o_lnp"})

In [37]:
topic_info=topic_info[['Topic', 'Count', 'Name','Representative_Docs', 'Representation', 'flan_snp',
       'openai4m_snp', 'openai4o_snp', 'flan_lnp', 'openai4m_lnp',
       'openai4o_lnp']]

In [38]:
#$topic_model.save('../results/topic_model/basic_model')
topic_info.to_excel('../results/topic_model/topic_info.xlsx',index=False)

## Stability

In [22]:
#base model
topic_model = BERTopic(verbose=True,embedding_model=sentence_model, low_memory=True, calculate_probabilities=False, 
                       vectorizer_model=vectorizer_model, 
                       hdbscan_model=hdbscan_model,
                       umap_model=umap_model,
                       ctfidf_model=ctfidf_model)

In [23]:
def get_topic_info_clean(topic_model):
    topic_info= topic_model.get_topic_info()
    
    topic_info['Aspect1'] = topic_info['Aspect1'].apply(lambda x: x[0] if x else '').str.replace("'", "").str.replace('"', '')
    topic_info['Aspect2'] = topic_info['Aspect2'].apply(lambda x: x[0] if x else '').str.replace("'", "").str.replace('"', '')
    topic_info['Aspect3'] = topic_info['Aspect3'].apply(lambda x: x[0] if x else '').str.replace("'", "").str.replace('"', '')
    topic_info['Aspect4'] = topic_info['Aspect4'].apply(lambda x: x[0] if x else '').str.replace("'", "").str.replace('"', '')
    topic_info['Aspect5'] = topic_info['Aspect5'].apply(lambda x: x[0] if x else '').str.replace("'", "").str.replace('"', '')
    topic_info['Aspect6'] = topic_info['Aspect6'].apply(lambda x: x[0] if x else '').str.replace("'", "").str.replace('"', '')
    
    topic_info= topic_info.rename(columns={
        "Aspect1": "flan_snp",
        "Aspect2": "flan_lnp",
        "Aspect3": "openai4m_snp",
        "Aspect4": "openai4m_lnp",
        "Aspect5": "openai4o_snp",
        "Aspect6": "openai4o_lnp"})
    
    topic_info=topic_info[['Topic', 'Count', 'Name','Representative_Docs', 'Representation', 'flan_snp',
           'openai4m_snp', 'openai4o_snp', 'flan_lnp', 'openai4m_lnp',
           'openai4o_lnp']]
    return topic_info


In [24]:
# Add all models together to be run in a single `fit`
representation_models = {
   "Main": main_representation,
    "Aspect1": flan_snp,
    "Aspect2": flan_lnp,
    "Aspect3": openai4m_snp,
    "Aspect4": openai4m_lnp,
    "Aspect5": openai4o_snp,
    "Aspect6": openai4o_lnp
}


In [25]:
%%time
topics, probabilities  = topic_model.fit_transform(text_df.text.values, embeddings)

2024-08-05 21:37:48,091 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-08-05 21:38:10,891 - BERTopic - Dimensionality - Completed ✓
2024-08-05 21:38:10,893 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-05 21:38:12,567 - BERTopic - Cluster - Completed ✓
2024-08-05 21:38:12,571 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-08-05 21:38:14,586 - BERTopic - Representation - Completed ✓


CPU times: user 28.2 s, sys: 565 ms, total: 28.8 s
Wall time: 28.1 s


In [69]:
topic_model.save('../results/topic_model/basic_model')

2024-08-05 14:58:25,554 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [26]:
ti_list = []

In [ ]:
for i in tqdm(range(4,21)):
    topic_model.load('../results/topic_model/basic_model') # I ensure I have the same basic model
    topic_model.update_topics(text_df.text.values,representation_model=representation_models)
    topic_info = get_topic_info_clean(topic_model)
    topic_info['iteration'] = i
    ti_list.append(topic_info)

  0%|          | 0/17 [00:00<?, ?it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [00:13<00:00,  7.61it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [00:16<00:00,  6.36it/s]

  0%|                                                                                                                                                                                                                                                   | 0/106 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [02:40<00:00,  1.51s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [02:41<00:00,  1.53s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [02:40<00:00,  1.51s/it]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [ ]:
topic_info_df = pd.concat(ti_list, ignore_index=True)

In [ ]:
previous_iterations  = pd.read_excel('../results/topic_model/topic_info_iteration.xlsx')

In [ ]:
all_iterations = pd.concat([previous_iterations,topic_info_df])

In [ ]:
all_iterations.to_excel('../results/topic_model/topic_info_iteration2.xlsx',index=False)

In [97]:
# topic_info_df.to_excel('../results/topic_model/topic_info_iteration.xlsx',index=False)